In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
'get_around_delay_analysis.xlsx'

'get_around_delay_analysis.xlsx'

In [3]:
df_delay = pd.read_excel('get_around_delay_analysis.xlsx', index_col=None)

In [4]:
df_delay.head(10)

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN
5,511626,398802,mobile,ended,-203.0,NaN,NaN
6,511639,370585,connect,ended,-15.0,563782.0,570.0
7,512303,371242,mobile,ended,-44.0,NaN,NaN
8,512475,322502,mobile,canceled,NaN,NaN,NaN
9,513434,256528,connect,ended,23.0,NaN,NaN


In [5]:
df_delay.sort_values(by = 'delay_at_checkout_in_minutes', ascending= False)

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
1021,532240,379042,mobile,ended,71084.0,NaN,NaN
1787,549980,402829,mobile,ended,64059.0,NaN,NaN
13776,537546,408750,mobile,ended,37382.0,NaN,NaN
1038,534699,270195,mobile,ended,23126.0,NaN,NaN
5257,556557,413591,mobile,ended,22618.0,NaN,NaN
...,...,...,...,...,...,...,...
21291,570189,257207,mobile,ended,NaN,NaN,NaN
21297,571481,311841,mobile,canceled,NaN,NaN,NaN
21300,572293,355023,connect,canceled,NaN,NaN,NaN
21302,573285,394863,mobile,ended,NaN,NaN,NaN


In [6]:
mask = df_delay['delay_at_checkout_in_minutes'] > 0
df_delay['checkout_status'] = ["Late" if x>0 else "in_time" for x in df_delay['delay_at_checkout_in_minutes']] 
print('Pourcentage de location rendu en retard:')
print(len(df_delay.loc[mask, 'delay_at_checkout_in_minutes'])/len(df_delay)*100)
fig = px.pie(df_delay, names='checkout_status', title='Proportion of late checkout')

Pourcentage de location rendu en retard:
44.12951665884562


In [7]:
fig.show()

In [8]:
print('Délai de checkout moyen:')
(df_delay.loc[:,'delay_at_checkout_in_minutes']).mean()

Délai de checkout moyen:


59.70151719075003

In [9]:
print('Temps de retard maximum:')
(df_delay.loc[:, 'delay_at_checkout_in_minutes']).max()

Temps de retard maximum:


71084.0

In [10]:
print('Temps de retard moyen lors de retard:')
(df_delay.loc[mask, 'delay_at_checkout_in_minutes']).mean()

Temps de retard moyen lors de retard:


201.79147171416417

In [11]:
print('Délai moyen entre deux locations:')
(df_delay.loc[:,'time_delta_with_previous_rental_in_minutes']).mean()

Délai moyen entre deux locations:


279.28843020097776

In [12]:
df_loc_consecutive = pd.merge(df_delay, df_delay, how='inner', left_on = 'previous_ended_rental_id', right_on = 'rental_id')

In [13]:
df_loc_consecutive

,rental_id_x,car_id_x,checkin_type_x,state_x,delay_at_checkout_in_minutes_x,previous_ended_rental_id_x,time_delta_with_previous_rental_in_minutes_x,checkout_status_x,rental_id_y,car_id_y,checkin_type_y,state_y,delay_at_checkout_in_minutes_y,previous_ended_rental_id_y,time_delta_with_previous_rental_in_minutes_y,checkout_status_y
0,511639,370585,connect,ended,-15.0,563782.0,570.0,in_time,563782,370585,connect,ended,136.0,NaN,NaN,Late
1,519491,312389,mobile,ended,58.0,545639.0,420.0,Late,545639,312389,mobile,ended,140.0,NaN,NaN,Late
2,521156,392479,mobile,ended,NaN,537298.0,0.0,in_time,537298,392479,mobile,ended,NaN,NaN,NaN,in_time
3,525044,349751,mobile,ended,NaN,510607.0,60.0,in_time,510607,349751,mobile,ended,-113.0,NaN,NaN,in_time
4,528808,181625,connect,ended,-76.0,557404.0,330.0,in_time,557404,181625,connect,ended,-352.0,554077.0,120.0,in_time
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1836,567741,294059,mobile,ended,111.0,567708.0,120.0,Late,567708,294059,mobile,ended,283.0,NaN,NaN,Late
1837,568241,396409,mobile,canceled,NaN,566136.0,570.0,in_time,566136,396409,mobile,ended,16.0,NaN,NaN,Late
1838,568523,297973,mobile,ended,12.0,567121.0,240.0,Late,567121,297973,mobile,ended,-275.0,NaN,NaN,in_time
1839,569717,377312,mobile,ended,230.0,545045.0,90.0,Late,545045,377312,mobile,ended,22.0,563963.0,630.0,Late


In [14]:
df_loc_consecutive.head()

,rental_id_x,car_id_x,checkin_type_x,state_x,delay_at_checkout_in_minutes_x,previous_ended_rental_id_x,time_delta_with_previous_rental_in_minutes_x,checkout_status_x,rental_id_y,car_id_y,checkin_type_y,state_y,delay_at_checkout_in_minutes_y,previous_ended_rental_id_y,time_delta_with_previous_rental_in_minutes_y,checkout_status_y
0,511639,370585,connect,ended,-15.0,563782.0,570.0,in_time,563782,370585,connect,ended,136.0,NaN,NaN,Late
1,519491,312389,mobile,ended,58.0,545639.0,420.0,Late,545639,312389,mobile,ended,140.0,NaN,NaN,Late
2,521156,392479,mobile,ended,NaN,537298.0,0.0,in_time,537298,392479,mobile,ended,NaN,NaN,NaN,in_time
3,525044,349751,mobile,ended,NaN,510607.0,60.0,in_time,510607,349751,mobile,ended,-113.0,NaN,NaN,in_time
4,528808,181625,connect,ended,-76.0,557404.0,330.0,in_time,557404,181625,connect,ended,-352.0,554077.0,120.0,in_time


In [15]:
df_loc_consecutive.drop(
    [
        "delay_at_checkout_in_minutes_x",
        "rental_id_y", 
        "car_id_y", 
        "state_y",
        "time_delta_with_previous_rental_in_minutes_y",
        "previous_ended_rental_id_y",
        "checkout_status_x"
    ], 
    axis=1,
    inplace=True
)

In [16]:
df_loc_consecutive.head()

,rental_id_x,car_id_x,checkin_type_x,state_x,previous_ended_rental_id_x,time_delta_with_previous_rental_in_minutes_x,checkin_type_y,delay_at_checkout_in_minutes_y,checkout_status_y
0,511639,370585,connect,ended,563782.0,570.0,connect,136.0,Late
1,519491,312389,mobile,ended,545639.0,420.0,mobile,140.0,Late
2,521156,392479,mobile,ended,537298.0,0.0,mobile,NaN,in_time
3,525044,349751,mobile,ended,510607.0,60.0,mobile,-113.0,in_time
4,528808,181625,connect,ended,557404.0,330.0,connect,-352.0,in_time


In [17]:
df_loc_consecutive.columns = [
    'rental_id',
    'car_id',
    'checkin_type',
    'state',
    'previous_ended_rental_id',
    'time_delta_with_previous_rental_in_minutes',
    'previous_checkin_type',
    'previous_delay_at_checkout_in_minutes',
    "previous_checkout_status"
]

In [18]:
df_loc_consecutive.head()

,rental_id,car_id,checkin_type,state,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,previous_checkin_type,previous_delay_at_checkout_in_minutes,previous_checkout_status
0,511639,370585,connect,ended,563782.0,570.0,connect,136.0,Late
1,519491,312389,mobile,ended,545639.0,420.0,mobile,140.0,Late
2,521156,392479,mobile,ended,537298.0,0.0,mobile,NaN,in_time
3,525044,349751,mobile,ended,510607.0,60.0,mobile,-113.0,in_time
4,528808,181625,connect,ended,557404.0,330.0,connect,-352.0,in_time


In [19]:
print("Percentage of missing values: ")
display(len(df_loc_consecutive))

Percentage of missing values: 


1841

In [20]:
df_loc_consecutive.head()

,rental_id,car_id,checkin_type,state,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,previous_checkin_type,previous_delay_at_checkout_in_minutes,previous_checkout_status
0,511639,370585,connect,ended,563782.0,570.0,connect,136.0,Late
1,519491,312389,mobile,ended,545639.0,420.0,mobile,140.0,Late
2,521156,392479,mobile,ended,537298.0,0.0,mobile,NaN,in_time
3,525044,349751,mobile,ended,510607.0,60.0,mobile,-113.0,in_time
4,528808,181625,connect,ended,557404.0,330.0,connect,-352.0,in_time


In [21]:
df_loc_consecutive = df_loc_consecutive[~df_loc_consecutive["previous_delay_at_checkout_in_minutes"].isnull()]
df_loc_consecutive.reset_index(drop=True, inplace=True)

In [22]:
mask2 = df_loc_consecutive["previous_delay_at_checkout_in_minutes"].notnull() 

In [23]:
df_loc_consecutive = df_loc_consecutive.loc[mask2, :]
df_loc_consecutive.reset_index(drop=True, inplace=True)

In [24]:
print('Nombre de location totale:{}'.format(len(df_delay)))
print('Nombre de location consécutive:{}'.format(len(df_loc_consecutive)))


Nombre de location totale:21310
Nombre de location consécutive:1729


In [25]:
df_loc_consecutive['delay_between_loc_in_min'] = df_loc_consecutive['time_delta_with_previous_rental_in_minutes'] - df_loc_consecutive['previous_delay_at_checkout_in_minutes']

In [26]:
df_loc_consecutive.head()

,rental_id,car_id,checkin_type,state,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,previous_checkin_type,previous_delay_at_checkout_in_minutes,previous_checkout_status,delay_between_loc_in_min
0,511639,370585,connect,ended,563782.0,570.0,connect,136.0,Late,434.0
1,519491,312389,mobile,ended,545639.0,420.0,mobile,140.0,Late,280.0
2,525044,349751,mobile,ended,510607.0,60.0,mobile,-113.0,in_time,173.0
3,528808,181625,connect,ended,557404.0,330.0,connect,-352.0,in_time,682.0
4,533670,320824,connect,ended,556563.0,630.0,connect,23.0,Late,607.0


In [27]:
12500/60/24

8.680555555555555

In [28]:
df_loc_consecutive.sort_values(by = 'delay_between_loc_in_min')

,rental_id,car_id,checkin_type,state,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,previous_checkin_type,previous_delay_at_checkout_in_minutes,previous_checkout_status,delay_between_loc_in_min
278,548388,297511,mobile,ended,546459.0,420.0,mobile,12968.0,Late,-12548.0
1535,564699,415572,mobile,ended,565228.0,240.0,mobile,4258.0,Late,-4018.0
512,541178,392635,mobile,ended,523175.0,0.0,mobile,2976.0,Late,-2976.0
1082,573931,300377,mobile,ended,568712.0,120.0,mobile,1760.0,Late,-1640.0
650,545099,389599,mobile,ended,539042.0,0.0,mobile,1500.0,Late,-1500.0
...,...,...,...,...,...,...,...,...,...,...
1570,557930,400886,mobile,ended,553320.0,450.0,mobile,-1414.0,in_time,1864.0
553,567806,216701,mobile,canceled,567644.0,690.0,mobile,-1408.0,in_time,2098.0
1628,564111,369648,connect,ended,561113.0,420.0,connect,-2109.0,in_time,2529.0
1198,570888,265239,connect,ended,551401.0,270.0,connect,-2938.0,in_time,3208.0


In [29]:
df_impacted_loc = df_loc_consecutive.loc[df_loc_consecutive['delay_between_loc_in_min'] < 0, :]
df_impacted_canceled_loc = df_impacted_loc.loc[df_impacted_loc['state'] == 'canceled',:]

In [30]:
no_late_cancel = len(df_loc_consecutive.loc[(df_loc_consecutive['previous_checkout_status'] == 'in_time') & (df_loc_consecutive['state'] == 'canceled')])
total_cancel = len(df_loc_consecutive.loc[(df_loc_consecutive['state'] == 'canceled')])

In [31]:
total_cancel

206

In [32]:
df_non_impacted_loc = df_loc_consecutive.loc[df_loc_consecutive['delay_between_loc_in_min'] >= 0, :]

In [33]:
no_late_cancel = len(df_non_impacted_loc.loc[(df_non_impacted_loc['state'] == 'canceled')])
print("Pourcentage d'annulation sans raisons apparentes: {}".format(no_late_cancel/len(df_non_impacted_loc) *100))


Pourcentage d'annulation sans raisons apparentes: 11.184645929847784


In [34]:
print("Pourcentage d'annulation hors retard: {}".format(len(df_delay)))

Pourcentage d'annulation hors retard: 21310


In [35]:
print('Nombre de location impactées: {}'.format(len(df_impacted_loc)))
print('Nombre de location annulées présentant un retard: {}'.format(len(df_impacted_canceled_loc)))
print("Pourcentage de location annulées lors de retard: {}".format(len(df_impacted_canceled_loc)/len(df_impacted_loc)*100))
print("On remarque un pourcentage de location annulées supérieurs lorsque le retard d'une location impact la location suivante")
print("On peut estimer le bombre de locaton annulées dû à ce retard à: {} (nombre de location impactées par retard moins le pourcentage de location annulées sans raison)".format(round(len(df_impacted_canceled_loc) - (len(df_impacted_canceled_loc)*17/100))))

Nombre de location impactées: 218
Nombre de location annulées présentant un retard: 37
Pourcentage de location annulées lors de retard: 16.972477064220186
On remarque un pourcentage de location annulées supérieurs lorsque le retard d'une location impact la location suivante
On peut estimer le bombre de locaton annulées dû à ce retard à: 31 (nombre de location impactées par retard moins le pourcentage de location annulées sans raison)


In [36]:
len(df_impacted_canceled_loc)/len(df_impacted_loc)*100

16.972477064220186

In [37]:
percent_impacted_loc = len(df_impacted_loc)/len(df_delay) *100

In [38]:
percent_impacted_loc

1.022993899577663

In [39]:
percent_cancel_late = (round(len(df_impacted_canceled_loc) - (len(df_impacted_canceled_loc)*(len(df_impacted_canceled_loc)/len(df_impacted_loc)*100)/100)))/len(df_delay) *100

In [40]:
percent_cancel_late

0.14547160957297042

In [41]:
#graph
#threshold

In [42]:
type(int(df_impacted_loc['previous_ended_rental_id'].values[0]))

int

In [43]:
df_delay['rental_id'].values

array([505000, 507750, 508131, ..., 573791, 574852, 575056], dtype=int64)

In [44]:
df_impacted_loc['previous_ended_rental_id'] = df_impacted_loc['previous_ended_rental_id'].apply(lambda x: int(x))

C:\Users\flori\AppData\Local\Temp\ipykernel_9752\1527267604.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [45]:
df_impacted_loc.head()

,rental_id,car_id,checkin_type,state,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,previous_checkin_type,previous_delay_at_checkout_in_minutes,previous_checkout_status,delay_between_loc_in_min
10,540479,374684,mobile,ended,539751,0.0,mobile,3.0,Late,-3.0
11,541862,382364,mobile,ended,540607,0.0,mobile,1.0,Late,-1.0
38,559781,408776,mobile,ended,540868,0.0,mobile,26.0,Late,-26.0
66,574568,301512,mobile,ended,572909,0.0,mobile,13.0,Late,-13.0
70,535519,353425,connect,ended,533413,0.0,connect,4.0,Late,-4.0


In [46]:
type(df_impacted_loc['previous_ended_rental_id'].values[0])

numpy.int64

In [47]:
for elt in df_delay['rental_id']:
    if elt in (df_impacted_loc['previous_ended_rental_id'].values):
        df_delay.loc[elt,'checkout_status'] = 'late and impacting'

In [48]:
df_delay['checkout_status'].value_counts

<bound method IndexOpsMixin.value_counts of 0                    in_time
1                    in_time
2                       Late
3                    in_time
4                    in_time
                 ...        
567682    late and impacting
550671    late and impacting
545720    late and impacting
556043    late and impacting
561890    late and impacting
Name: checkout_status, Length: 21526, dtype: object>

In [49]:
df_delay.head(10)

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,checkout_status
0,505000.0,363965.0,mobile,canceled,NaN,NaN,NaN,in_time
1,507750.0,269550.0,mobile,ended,-81.0,NaN,NaN,in_time
2,508131.0,359049.0,connect,ended,70.0,NaN,NaN,Late
3,508865.0,299063.0,connect,canceled,NaN,NaN,NaN,in_time
4,511440.0,313932.0,mobile,ended,NaN,NaN,NaN,in_time
5,511626.0,398802.0,mobile,ended,-203.0,NaN,NaN,in_time
6,511639.0,370585.0,connect,ended,-15.0,563782.0,570.0,in_time
7,512303.0,371242.0,mobile,ended,-44.0,NaN,NaN,in_time
8,512475.0,322502.0,mobile,canceled,NaN,NaN,NaN,in_time
9,513434.0,256528.0,connect,ended,23.0,NaN,NaN,Late


In [50]:
fig = px.pie(df_delay, names='checkout_status', title='Proportion of late checkout')

In [51]:
fig.show()

In [52]:
df_loc_consecutive.head()

,rental_id,car_id,checkin_type,state,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,previous_checkin_type,previous_delay_at_checkout_in_minutes,previous_checkout_status,delay_between_loc_in_min
0,511639,370585,connect,ended,563782.0,570.0,connect,136.0,Late,434.0
1,519491,312389,mobile,ended,545639.0,420.0,mobile,140.0,Late,280.0
2,525044,349751,mobile,ended,510607.0,60.0,mobile,-113.0,in_time,173.0
3,528808,181625,connect,ended,557404.0,330.0,connect,-352.0,in_time,682.0
4,533670,320824,connect,ended,556563.0,630.0,connect,23.0,Late,607.0


In [53]:
df_loc_consecutive.sort_values(by = 'delay_between_loc_in_min')

,rental_id,car_id,checkin_type,state,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,previous_checkin_type,previous_delay_at_checkout_in_minutes,previous_checkout_status,delay_between_loc_in_min
278,548388,297511,mobile,ended,546459.0,420.0,mobile,12968.0,Late,-12548.0
1535,564699,415572,mobile,ended,565228.0,240.0,mobile,4258.0,Late,-4018.0
512,541178,392635,mobile,ended,523175.0,0.0,mobile,2976.0,Late,-2976.0
1082,573931,300377,mobile,ended,568712.0,120.0,mobile,1760.0,Late,-1640.0
650,545099,389599,mobile,ended,539042.0,0.0,mobile,1500.0,Late,-1500.0
...,...,...,...,...,...,...,...,...,...,...
1570,557930,400886,mobile,ended,553320.0,450.0,mobile,-1414.0,in_time,1864.0
553,567806,216701,mobile,canceled,567644.0,690.0,mobile,-1408.0,in_time,2098.0
1628,564111,369648,connect,ended,561113.0,420.0,connect,-2109.0,in_time,2529.0
1198,570888,265239,connect,ended,551401.0,270.0,connect,-2938.0,in_time,3208.0


In [54]:
min(df_loc_consecutive['delay_between_loc_in_min'])

-12548.0

In [55]:
df_price = pd.read_csv('get_around_pricing_project.csv')

In [56]:
df_price.head()

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [57]:
df_impacted_canceled_loc.head()

,rental_id,car_id,checkin_type,state,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,previous_checkin_type,previous_delay_at_checkout_in_minutes,previous_checkout_status,delay_between_loc_in_min
122,539151,282893,mobile,canceled,548646.0,30.0,mobile,201.0,Late,-171.0
149,553139,297851,mobile,canceled,547650.0,240.0,mobile,410.0,Late,-170.0
162,559037,349171,connect,canceled,552392.0,0.0,connect,46.0,Late,-46.0
175,543745,359045,connect,canceled,521852.0,150.0,connect,650.0,Late,-500.0
196,538224,340014,connect,canceled,534999.0,180.0,connect,346.0,Late,-166.0


In [58]:
df_impacted_canceled_loc = df_impacted_canceled_loc.reset_index()

In [59]:
df_impacted_canceled_loc = df_impacted_canceled_loc.drop('index', axis = 1)

In [60]:
df_impacted_canceled_loc.head()

,rental_id,car_id,checkin_type,state,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,previous_checkin_type,previous_delay_at_checkout_in_minutes,previous_checkout_status,delay_between_loc_in_min
0,539151,282893,mobile,canceled,548646.0,30.0,mobile,201.0,Late,-171.0
1,553139,297851,mobile,canceled,547650.0,240.0,mobile,410.0,Late,-170.0
2,559037,349171,connect,canceled,552392.0,0.0,connect,46.0,Late,-46.0
3,543745,359045,connect,canceled,521852.0,150.0,connect,650.0,Late,-500.0
4,538224,340014,connect,canceled,534999.0,180.0,connect,346.0,Late,-166.0


In [61]:
fig2= px.bar(df_impacted_canceled_loc, x= 'delay_between_loc_in_min', y = df_impacted_canceled_loc.index )

In [62]:
fig2.show()

In [63]:
for i in range(df_loc_consecutive['delay_between_loc_in_min'].min(),df_loc_consecutive['delay_between_loc_in_min'].max(), 100):
    

SyntaxError: incomplete input (933871700.py, line 2)

In [ ]:
df_loc_consecutive.loc

In [64]:
df_price = pd.read_csv('get_around_pricing_project.csv')

In [65]:
df_price

,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4838,4838,Toyota,39743,110,diesel,black,van,False,True,False,False,False,False,True,121
4839,4839,Toyota,49832,100,diesel,grey,van,False,True,False,False,False,False,True,132
4840,4840,Toyota,19633,110,diesel,grey,van,False,True,False,False,False,False,True,130
4841,4841,Toyota,27920,110,diesel,brown,van,True,True,False,False,False,False,True,151
